In [ ]:
pip install transformers

In [4]:
pip install torchaudio

In [12]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import torchaudio
import torch

In [3]:
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h").to("cpu")
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You sho

preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

In [10]:
def tokenize_audio(audio_file_path):
    # Load the audio file
    waveform, sample_rate = torchaudio.load(audio_file_path)

    # Resample the audio file if necessary
    if sample_rate != processor.feature_extractor.sampling_rate:
        resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=processor.feature_extractor.sampling_rate)
        waveform = resampler(waveform)

    # Tokenize the waveform
    inputs = processor(waveform.squeeze(0), sampling_rate=processor.feature_extractor.sampling_rate, return_tensors="pt", padding=True)

    return inputs

# Replace 'path_to_your_audio_file.wav' with the path to the audio file you want to process
audio_file_path = '/content/common_voice_en_1.mp3'
tokenized_inputs = tokenize_audio(audio_file_path)

# To get the output tokens as integers
tokens = tokenized_inputs.input_values[0].tolist()

# Print the tokens
print(tokens)

[0.0002679597819224, 0.0002679597819224, 0.0002679597819224, 0.0002679597819224, 0.0002679597819224, 0.0002679597819224, 0.0002679597819224, 0.0002679597819224, 0.0002679597819224, 0.0002679597819224, 0.0002679597819224, 0.0002679597819224, 0.0002679597819224, 0.0002679597819224, 0.0002679597819224, 0.0002679597819224, 0.0002679597819224, 0.0002679597819224, 0.0002679597819224, 0.0002679597819224, 0.0002679597819224, 0.0002679597819224, 0.0002679597819224, 0.0002679597819224, 0.0002679597819224, 0.0002679597819224, 0.0002679597819224, 0.0002679597819224, 0.0002679597819224, 0.0002679597819224, 0.0002679597819224, 0.0002679597819224, 0.0002679597819224, 0.0002679597819224, 0.0002679597819224, 0.0002679597819224, 0.0002679597819224, 0.0002679597819224, 0.0002679597819224, 0.0002679597819224, 0.0002679597819224, 0.0002679597819224, 0.0002679597819224, 0.0002679597819224, 0.0002679597819224, 0.0002679597819224, 0.0002679597819224, 0.0002679597819224, 0.0002679597819224, 0.0002679597819224,

In [13]:
# Perform the prediction
with torch.no_grad():
    logits = model(tokenized_inputs.input_values).logits

# Retrieve the predicted tokens
predicted_ids = torch.argmax(logits, dim=-1)

# Decode the tokens to text
transcription = processor.batch_decode(predicted_ids)

# Print the transcription
print(transcription)


["I'M INTERESTED ONLY IN THE PRESENT"]
